 # 데이터는 Data.zip에서 받으세요.
 
 # Quant.py는 beginner8 알집파일

In [1]:
cd C:\Users\USER\Quant

C:\Users\USER\Quant


In [2]:
import Quant as Quant
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
path = '재무데이터.xlsx'
path_price = '수정주가.csv'
path_mkt = '시총데이터.csv'
path_endmonth = '결산월.csv'
path_kospiyn = '거래소데이터.csv'
path_delist_and_stop = '상장폐지데이터.csv'

In [4]:
cleaned_data = Quant.preprocessing_path(path)
cleaned_price = Quant.preprocessing_price(path_price)
cleaned_kospiyn = Quant.preprocessing_kospiyn(path_kospiyn)
cleaned_mkt = Quant.preprocessing_mktdata(path_mkt)
delist_data, stop_data = Quant.preprocessing_stop_and_delist(path_delist_and_stop)
cleaned_endmonth = Quant.preprocessing_period(path_endmonth)

 ### 문제 1. 전처리 수정주가데이터와 상장폐지 데이터를 사용하여 다음 조건에 맞는 함수를 만드시오.
 
 ##### 지금 수정주가의 경우 상장폐지 이후 가격이 NaN으로 나타나고 있습니다. 
 ##### 상장폐지된 종목들의 상장폐지 이후 가격을 0으로 만들어주는 함수를 만드시오.
 price_after_delist(cleaned_price, delist_data) 함수를 실행할 경우 
 상장폐지 이후가격이 0으로 처리된 수정주가 데이터가 나오도록 함수를 만드세요.

반복문을 사용해도 됩니다.

def price_after_delist(cleaned_price , delist_data) :

    #################################
    ## 상장폐지 이후 가격이 0이 됨 ##
    #################################
    
    return p

In [6]:
def price_after_delist(cleaned_price, delist_data) :
    cleaned_price2 = cleaned_price.copy()
    x1_list = delist_data.index
    x2_list = list(delist_data[delist_data.columns[0]])    
    
    lst = []
    for i in range(len(x1_list)) :
        x1, x2 = x1_list[i], x2_list[i]
        cleaned_price2[x1][x2:] = np.zeros(len(cleaned_price2[x1][x2:]))
        lst.append(cleaned_price2[x1])
    p1 = cleaned_price2[cleaned_price2.columns.difference(x1_list)]
    p2 = pd.concat(lst,axis = 1)
    p = pd.concat([p1,p2],axis = 1)
    return p[cleaned_price2.columns]

In [7]:
price_after_delist(cleaned_price, delist_data).head(4)

,A005930,A000660,A207940,A035420,A068270,A051910,A035720,A006400,A005380,A036570,...,A192520,A192530,A334890,A350520,A357250,A900030,A900050,A950010,A950070,A950100
Symbol,,,,,,,,,,,,,,,,,,,,,
1980-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-04,79.0,NaN,NaN,NaN,NaN,NaN,NaN,4776.0,2416.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 ### 문제 2. 현재 수정주가 일 수익률이 300%인 극단적 종목들을 제외하려고한다. 
 #### 다음 가이드라인에 따라 함수를 만드세요.
 
def preprocessing_backtest(cleaned_data, cleaned_price, cleaned_mkt, delist_data, stop_data) :

     1. extreme = 전체 종목중에 1일동안 주식 수익률이 300%가 넘어가는 극단적 종목 인덱스
     2. not_extreme = 전체 종목중 extreme종목을 제외한 종목명(인덱스형식)
     3. STOP = not_extreme에 해당되는 종목들의 거래정지 데이터를 Monthly resample로 나타낸 데이터프레임.
         STOP 데이터의 경우 이중컬럼을 제거하고 cleaned_price처럼 index 는 날짜 column는 종목 구성의 데이터프레임으로 나타내시오.
     4. P = not_extreme에 해당되는 종목들의 수정주가 데이터를 Monthly resample로 나타낸 데이터프레임.
     5. DELIST = 상장폐지 데이터에서 not_extreme에 해당하는 종목만 존재하는 상장폐지데이터
     6. PRICE = 4번 과정을 처리하고, 문제 1의 함수를 실행한 데이터 : 즉, PRICE = price_after_delist(P,DELIST) 
     7. MKT = not_extreme에 해당되는 종목들의 시가총액 데이터를 Monthly resample로 나타낸 데이터프레임.
     8. FINANCE = 재무데이터에서 extreme_index 제외
     return FINANCE, PRICE, MKT, DELIST, STOP

In [12]:
def preprocessing_backtest(cleaned_data, cleaned_price, cleaned_mkt, 
                           delist_data, stop_data) :
    extreme_return = (cleaned_price.pct_change() > 3).max(0)
    not_extreme = extreme_return[extreme_return == False].index
    P = cleaned_price[not_extreme].resample('M').last()
    STOP = stop_data['거래정지여부'].loc[not_extreme].T.resample('M').last()
    DELIST = delist_data.loc[delist_data.index.intersection(not_extreme)]
    FINANCE = cleaned_data.loc[not_extreme]
    PRICE = price_after_delist(P, DELIST)
    MKT = cleaned_mkt[not_extreme].resample('M').last()
    return FINANCE, PRICE, MKT, DELIST, STOP

In [13]:
FINANCE, PRICE, MKT, DELIST, STOP = preprocessing_backtest(cleaned_data, cleaned_price, cleaned_mkt, 
                           delist_data, stop_data)

### 문제 3. 

어떤 Timestamp데이터의 month가 5보다 작거나같으면(<=) data_Q = 제작년 12월 31일을 리턴하고, 

그 이외의 경우에는 data_Q = 작년 12월 31일을 리턴하는 함수를 만드시오.

(2001년 이전에는 6월은 돼야지 작년 재무데이터가 나옴)

def calculate_data_Q_before_2001(rebalance_day) :

    ##################################################################
    ### rebalance_day.month가 5보다 작으면 제작년 12월 31일 리턴   ###
    ### 그렇지 않으면 작년 12월 31일을 리턴                        ###
    ##################################################################

    return pd.to_datetime(data_Q)
   
calculate_data_Q_before_2001(pd.to_datetime('1998-06-10')) 결과 >> pd.Timestamp(1997년 12월 31일)
calculate_data_Q_before_2001(pd.to_datetime('1998-04-15')) 결과 >> pd.Timestamp(1996년 12월 31일)

In [22]:
def calculate_data_Q_before_2001(rebalance_day):
    if rebalance_day.month <= 5 :
        data_Q = str(rebalance_day.year - 2) + '-12-31'
    else :
        data_Q = str(rebalance_day.year - 1) + '-12-31'
    return pd.to_datetime(data_Q)

In [23]:
calculate_data_Q_before_2001(pd.to_datetime('1998-06-01'))

Timestamp('1997-12-31 00:00:00')

In [24]:
calculate_data_Q_before_2001(pd.to_datetime('1998-03-01'))

Timestamp('1996-12-31 00:00:00')

### 문제 4. 

4분기 재무데이터는 직전 12월 31일로부터 121일 뒤에 모두 이용가능하다, 

1분기,2분기,3분기 재무데이터는 각각 3월 31일/ 6월 30일/ 9월 30일로부터 76일 뒤에 이용가능하다

특정 날짜를 rebalance_day라고 가정하자. 

다음 조건에 따라 이용가능한 최근분기 날짜를 계산하는 함수를 만드시오. 

if rebalance_day <= 직전4분기 데이터 공시날짜:

    return data_Q = 이전년도 09월 30일

elif rebalance_day <= 1분기 데이터 공시날짜 and rebalance_day >= 직전4분기 데이터 공시날짜 :

    return data_Q = 이전년도 12월 31일

elif rebalance_day <= 2분기 데이터 공시날짜 and rebalance_day >= 당해1분기 데이터 공시날짜 :

    return data_Q = 당해 3월 31일

elif rebalance_day <= 3분기 데이터 공시날짜 and rebalance_day >= 당해 2분기 데이터 공시날짜 :

    return data_Q = 당해 6월 30일

else :

    return data_Q = 당해 9월 30일

def calculate_data_Q_after_2001(rebalance_day) :

    ##################################################################
    ### 위의 조건에 따라 data_Q를 리턴하는 함수를 만드시오         ###
    ###                                                            ###
    ##################################################################

    return pd.to_datetime(data_Q)
    
calculate_date_Q_after_2001(pd.to_datetime('2005-01-25')) >>> 결과로 2004년 9월 30일

calculate_date_Q_after_2001(pd.to_datetime('2005-05-25')) >>> 결과로 2004년 12월 31일

calculate_date_Q_after_2001(pd.to_datetime('2005-06-25')) >>> 결과로 2005년 3월 31일

calculate_date_Q_after_2001(pd.to_datetime('2005-09-25')) >>> 결과로 2005년 6월 30일

calculate_date_Q_after_2001(pd.to_datetime('2005-12-23')) >>> 결과로 2005년 9월 30일


In [26]:
def calculate_data_Q_after_2001(rebalance_day):
    year = rebalance_day.year
    bQ3 = pd.to_datetime(str(year-1) + '-09-30')
    bQ4 = pd.to_datetime(str(year-1)+'-12-31')
    Q1 = pd.to_datetime(str(year)+'-03-31')
    Q2 = pd.to_datetime(str(year)+'-06-30')
    Q3 = pd.to_datetime(str(year)+'-09-30')
    bQ4_report_day = bQ4 + relativedelta(days = 121)
    Q1_report_day = Q1 + relativedelta(days = 76)
    Q2_report_day = Q2 + relativedelta(days = 76)
    Q3_report_day = Q3 + relativedelta(days = 76)
    if rebalance_day <= bQ4_report_day :
        data_Q = bQ3
    elif rebalance_day > bQ4_report_day and rebalance_day <= Q1_report_day :
        data_Q = bQ4
    elif rebalance_day > Q1_report_day and rebalance_day <= Q2_report_day :
        data_Q = Q1
    elif rebalance_day > Q2_report_day and rebalance_day <= Q3_report_day :
        data_Q = Q2
    else :        
        data_Q = Q3
    return data_Q

In [27]:
calculate_data_Q_after_2001(rebalance_day = pd.to_datetime('2005-02-01'))

Timestamp('2004-09-30 00:00:00')

In [32]:
calculate_data_Q_after_2001(rebalance_day = pd.to_datetime('2005-05-08'))

Timestamp('2004-12-31 00:00:00')

In [31]:
calculate_data_Q_after_2001(rebalance_day = pd.to_datetime('2005-08-04'))

Timestamp('2005-03-31 00:00:00')

 ### 문제 5. 
 
 rebalance_day의 날짜가 2001년 5월 1일 이전이라면 
 
 data_Q = calculate_data_Q_before_2001(rebalance_day)
 
 그렇지 않으면
 
 data_Q = calculate_data_Q_after_2001(rebalance_day)를 리턴하는 함수를 만드세요.

In [20]:
def calculate_data_Q(rebalance_day) :
    rebalance_day = pd.to_datetime(rebalance_day)
    if rebalance_day < pd.to_datetime('2001-05-01') :
        data_Q = calculate_data_Q_before_2001(rebalance_day)
    else :
        data_Q = calculate_data_Q_after_2001(rebalance_day)
    return data_Q

In [33]:
calculate_data_Q(rebalance_day = pd.to_datetime('2005-08-04'))

Timestamp('2005-03-31 00:00:00')

In [34]:
calculate_data_Q(rebalance_day = pd.to_datetime('1998-08-04'))

Timestamp('1997-12-31 00:00:00')